In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Imports


In [13]:

from bioreaction.simulation.manager import simulate_steady_states
from functools import partial
from typing import Optional, Tuple
import os
import sys
import numpy as np
import jax
import jax.numpy as jnp
import diffrax as dfx
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

pink_palette = sns.cubehelix_palette(rot=0.5, start=2.7, gamma=1.1)

jax.config.update('jax_platform_name', 'cpu')
# os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.70"

if __package__ is None:

    module_path = os.path.abspath(os.path.join('..'))
    sys.path.append(module_path)

    __package__ = os.path.basename(module_path)


root_dir = '..'

In [14]:
from synbio_morpher.utils.circuit.agnostic_circuits.circuit_manager import CircuitModeller
from synbio_morpher.utils.common.setup import construct_circuit_from_cfg
from synbio_morpher.utils.evolution.evolver import Evolver
from synbio_morpher.utils.results.analytics.timeseries import generate_analytics
from synbio_morpher.utils.results.analytics.naming import get_analytics_types_all, get_true_names_analytics, get_true_interaction_cols
from synbio_morpher.utils.misc.type_handling import append_nest_dicts
from synbio_morpher.utils.misc.numerical import add_recursively
from synbio_morpher.utils.misc.helper import vanilla_return
from synbio_morpher.utils.modelling.deterministic import bioreaction_sim_dfx_expanded
from synbio_morpher.utils.data.data_format_tools.common import load_json_as_dict
from synbio_morpher.srv.io.manage.script_manager import script_preamble

# Find good starting circuits


## Load data


In [15]:
og_dir = '../data/ensemble_mutation_effect_analysis/2023_07_15_192831/generate_species_templates'
fn = '../data/ensemble_mutation_effect_analysis/2023_07_17_105328/summarise_simulation/tabulated_mutation_info.csv'
data = pd.read_csv(fn)

config = load_json_as_dict(os.path.join(fn.split('summarise')[
                           0], 'mutation_effect_on_interactions_signal', 'experiment.json'))
config = config['config_filepath']
config['experiment']['purpose'] = 'tests'

## Select starting candidate

We want to start with a circuit that has high adaptability, so a circuit that has high signal sensitivity and precision. We'll use the non-signal species $RNA_1$ and $RNA_2$ as the outputs.

One thing to look out for is that because precision is often then inverse of sensitivity, it will be much larger typically, so selecting the largest for both should give preference to sensitivity.


In [16]:
signal_species = config['signal']['inputs']

filt = (
    (data[get_true_interaction_cols(data, 'energies')].sum(axis=1) != 0) &
    (data['sample_name'].isin(signal_species) != True) &
    (data['overshoot'] > 0)
)

percentile = 0.9
sensitivity_range = data[filt]['sensitivity_wrt_species-6'] > (data[filt]['sensitivity_wrt_species-6'].max() *
                                                               percentile)
starting_circ_rows = data[filt][sensitivity_range].sort_values(
    by=['sensitivity_wrt_species-6', 'precision_wrt_species-6'], ascending=False)

print(
    f'Picking circuits that have a sensitivity of at least {data[filt]["sensitivity_wrt_species-6"].max() * percentile}')

starting_circ_rows

Picking circuits that have a sensitivity of at least 0.36505494


,circuit_name,mutation_name,mutation_num,mutation_type,mutation_positions,path_to_template_circuit,index,name,interacting,self_interacting,...,RMSE_diff_to_base_circuit,steady_states_diff_to_base_circuit,fold_change_ratio_from_mutation_to_base,initial_steady_states_ratio_from_mutation_to_base,max_amount_ratio_from_mutation_to_base,min_amount_ratio_from_mutation_to_base,overshoot_ratio_from_mutation_to_base,RMSE_ratio_from_mutation_to_base,steady_states_ratio_from_mutation_to_base,sample_name
1042709,toy_mRNA_circuit_11726,RNA_0_m3-8,3,"[5, 6, 4]","[15, 9, 4]",data/ensemble_mutation_effect_analysis/2023_07...,28.0,toy_mRNA_circuit_11726,[[0 2]\n [1 2]],[],...,2.967089,-2.952154,0.588115,0.512502,0.507635,0.287576,43.528206,inf,0.301410,RNA_2
377023,toy_mRNA_circuit_10622,RNA_0_m5-9,5,"[11, 2, 0, 0, 2]","[5, 12, 1, 15, 19]",data/ensemble_mutation_effect_analysis/2023_07...,39.0,toy_mRNA_circuit_10622,[[0 1]],[[1 1]],...,1.291806,-1.207342,0.582016,1.168085,1.168085,0.628673,inf,inf,0.679844,RNA_1
1099279,toy_mRNA_circuit_1182,RNA_1_m15-1,15,"[4, 10, 5, 5, 2, 2, 6, 7, 3, 4, 4, 7, 10, 10, 9]","[14, 4, 15, 17, 19, 0, 10, 11, 8, 1, 13, 9, 12...",data/ensemble_mutation_effect_analysis/2023_07...,111.0,toy_mRNA_circuit_1182,[[0 1]],[[1 1]],...,197.186770,-197.146770,0.606385,0.020804,0.020804,0.011972,inf,inf,0.012615,RNA_1
1458932,toy_mRNA_circuit_12416,RNA_2_m15-3,15,"[10, 10, 4, 8, 0, 7, 10, 6, 9, 7, 0, 10, 11, 4...","[3, 16, 5, 1, 7, 10, 4, 15, 0, 14, 17, 18, 8, ...",data/ensemble_mutation_effect_analysis/2023_07...,173.0,toy_mRNA_circuit_12416,[[0 2]],[[2 2]],...,66.485640,-66.276080,0.641676,0.072935,0.072935,0.040039,inf,inf,0.046801,RNA_2
1037908,toy_mRNA_circuit_11718,RNA_1_m10-7,10,"[8, 1, 1, 1, 4, 10, 1, 4, 7, 6]","[9, 8, 19, 16, 7, 11, 14, 5, 15, 1]",data/ensemble_mutation_effect_analysis/2023_07...,107.0,toy_mRNA_circuit_11718,[[0 1]],[[1 1]],...,197.259000,-197.124040,0.651164,0.019548,0.019548,0.011204,inf,inf,0.012729,RNA_1
440486,toy_mRNA_circuit_10728,RNA_0_m2-6,2,"[5, 1]","[8, 12]",data/ensemble_mutation_effect_analysis/2023_07...,16.0,toy_mRNA_circuit_10728,[[0 2]\n [1 2]],[[1 1]],...,7.839827,-7.709217,0.613702,0.313857,0.313857,0.158647,inf,inf,0.192615,RNA_2
741844,toy_mRNA_circuit_11227,RNA_0_m2-4,2,"[2, 7]","[5, 9]",data/ensemble_mutation_effect_analysis/2023_07...,14.0,toy_mRNA_circuit_11227,[[0 1]],[[1 1]],...,1.061432,-0.944925,0.645727,1.122054,1.122054,0.652472,inf,inf,0.724541,RNA_1
657892,toy_mRNA_circuit_11088,RNA_1_m3-5,3,"[3, 10, 0]","[14, 3, 12]",data/ensemble_mutation_effect_analysis/2023_07...,85.0,toy_mRNA_circuit_11088,[[0 1]],[[1 1]\n [2 2]],...,12.703225,-13.872738,0.932910,0.190074,0.190074,0.213711,0.066885,inf,0.177322,RNA_1
73457,toy_mRNA_circuit_10119,RNA_0_m3-9,3,"[11, 6, 3]","[11, 18, 16]",data/ensemble_mutation_effect_analysis/2023_07...,29.0,toy_mRNA_circuit_10119,[[0 2]\n [1 2]],[],...,8.245810,-8.097506,0.614938,0.393121,0.393121,0.208270,inf,inf,0.241745,RNA_2
73409,toy_mRNA_circuit_10119,RNA_0_m2-3,2,"[9, 5]","[11, 14]",data/ensemble_mutation_effect_analysis/2023_07...,13.0,toy_mRNA_circuit_10119,[[0 2]\n [1 2]],[],...,8.060954,-7.896498,0.626618,0.415831,0.415831,0.223564,inf,inf,0.260567,RNA_2


In [17]:
starting_circ_rows[['sensitivity_wrt_species-6', 'precision_wrt_species-6', 'overshoot']]

,sensitivity_wrt_species-6,precision_wrt_species-6,overshoot
1042709,0.405617,2.664728,0.070466
377023,0.392066,2.817703,0.192975
1099279,0.383694,2.810953,0.128534
1458932,0.382633,3.289007,0.470139
1037908,0.381519,3.206638,0.304456
440486,0.381112,3.358389,0.324340
741844,0.380047,3.107846,0.247221
657892,0.379448,3.055319,0.279528
73457,0.377291,3.236075,0.357479
73409,0.376741,3.286376,0.395164


In [ ]:

data['Starting circuit'] = (data['circuit_name'].isin(starting_circ_rows['circuit_name'])) & \
    (data['mutation_name'].isin(starting_circ_rows['mutation_name'])) & filt
fig = px.scatter_3d(
    data.sort_values(by='Starting circuit'), x='sensitivity_wrt_species-6', y='precision_wrt_species-6', z='overshoot',
    log_x=True, log_z=True, 
    color='Starting circuit', symbol=((data.sort_values(by='Starting circuit')['Starting circuit'] + 0.1)/1.1)
)

fig.show()
# sns.scatterplot(
#     data.sort_values(by='Starting circuit'), x='sensitivity_wrt_species-6', y='precision_wrt_species-6',
#     hue='Starting circuit', alpha=((data.sort_values(by='Starting circuit')['Starting circuit'] + 0.1)/1.1)
# )
# plt.xscale('log')
# plt.yscale('log')
# plt.xlabel('Sensitivity')
# plt.ylabel('Precision')

# Monte Carlo optimisation

We want to push circuits into ranges of higher sensitivity


## Optimisation function


In [20]:
def choose_next(batch: list, starting: list):
    batch_analytics = [circuit.result_collector.get_result('signal').analytics for circuit in batch]
    [circuit.result_collector.get_result('signal').analytics for circuit in batch]
    data = pd.DataFrame(
        data=
    )
    
    
    
    

## Optimisation loop


### Initialise and set up run

In [21]:
config, data_writer = script_preamble(config)

CircuitModeller(data_writer, config)
Evolver

simulator.py:check_IntaRNA_path():112: Could not detect IntaRNA on system: CompletedProcess(args='which IntaRNA', returncode=1, stdout='', stderr='') WARNING


synbio_morpher.utils.evolution.evolver.Evolver

### Start loop

In [22]:


total_steps = 1000

starting = 0
for step in range(total_steps):

    batch = mutate(starting)
    batch = simulate(batch)
    starting = choose_next(batch, starting)

NameError: name 'mutate' is not defined

## Visualise circuit trajectory
